In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from pathlib import Path
import numpy as np
import pandas as pd
import pickle
import json
import torch
from tqdm import tqdm
import os
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import seaborn as sns

methods = ['Oracle', 'ERM', 'GroupDRO', 'IRM', 'VREx', 'RVP','IGA', 'CORAL', 'MLDG']
es_mapping = {
    'train': "Model selection method: training domains",
    'val': "Model selection method: validation domain",
}

In [2]:
def load_final_results(folder):
    data = torch.load((folder.parent/'stats.pkl').open('rb')) 
    if 'es_roc' not in data:
        results = []
        lines = (folder.parent/'results.jsonl').open('r').readlines()
        for line in lines:
            results.append(json.loads(line))
        result_row = [i for i in results if i['step'] == data['es_step']][0]
        data['es_roc'] = result_row['es_roc']
    return data

In [3]:
def extract_stats(lst, keep):
    df_lst = []
    for i in lst:
        temp = {}
        for k in keep:
            if '/' in k:
                key1, key2 = k.split('/')
                if key2 in ['penalty_anneal_iters', 'lambda']:
                    temp[key2] = i[key1][i['args']['algorithm'].lower() +'_' + key2]
                else:                
                    temp[key2] = 'Oracle' if key2 == 'algorithm' and i[key1][key2] =='ERMID' else i[key1][key2]
                
            else:
                temp[k] = i[k]
        df_lst.append(temp)
    return pd.DataFrame(df_lst)   

def load_saved_data(path):
    for i in tqdm(path.glob('*/done')):
        lst.append(load_final_results(i))
    return lst

In [4]:
def agg_and_group(df, distinct_hparams, eval_metric, other_metrics):
    # best model for the particular trial
    df = df.groupby(distinct_hparams + ['trial_seed']).apply(lambda x: x.loc[x[eval_metric].idxmax(), [i for i in [eval_metric] + other_metrics]]).reset_index()
    # averaged over all 5 trials
    df = df.groupby(distinct_hparams).agg({i: ['mean', 'std'] for i in [eval_metric] + other_metrics}).reset_index()                                     
    df.columns = ["_".join(pair) if pair[1] != '' else pair[0] for pair in df.columns ]
    for i in [eval_metric] + other_metrics:
        df[i] = df[f'{i}_mean'].apply('{:.3f}'.format) + u"\u00B1" + df[f'{i}_std'].apply('{:.3f}'.format)
    return df

def pivot(df, values, index, methods = methods):
    temp = df.pivot_table(values = values, index = index, columns = ['algorithm'], aggfunc = lambda x: x)
    temp.columns = [pair[1] if pair[1] != '' else pair[0] for pair in temp.columns]
    temp = temp[list(temp.columns[:len(temp.columns) - len(methods)]) + methods]
    return temp

## eICU CorrLabel

In [49]:
path = Path('/scratch/hdd001/home/haoran/domainbed/eICUCorrLabel')
lst = load_saved_data(path)   

In [50]:
keep = ['es_roc', 'test_results/South_roc', 'test_results/South_tpr_gap_thres', 
        'model_hparams/eicu_architecture', 'args/algorithm', 'args/es_method', 'args/hparams_seed',
       'args/trial_seed', 'model_hparams/corr_label_train_corrupt_mean']

df = extract_stats(lst, keep)

In [51]:
distinct_hparams = ['algorithm', 'es_method', 'corr_label_train_corrupt_mean']
eval_metric = 'es_roc'
other_metrics = ['South_roc']

df = agg_and_group(df, distinct_hparams, eval_metric, other_metrics)

In [52]:
pivot(df, ['South_roc'], ['es_method', 'corr_label_train_corrupt_mean'])

Oracle          ERM  \
es_method corr_label_train_corrupt_mean                                
train     0.1                               0.958±0.008  0.327±0.035   
          0.3                               0.958±0.006  0.692±0.009   
          0.5                               0.958±0.008  0.863±0.009   
val       0.1                               0.958±0.008  0.716±0.047   
          0.3                               0.958±0.008  0.713±0.021   
          0.5                               0.958±0.008  0.865±0.005   

                                               GroupDRO          IRM  \
es_method corr_label_train_corrupt_mean                                
train     0.1                               0.333±0.045  0.340±0.023   
          0.3                               0.681±0.022  0.689±0.020   
          0.5                               0.864±0.009  0.869±0.002   
val       0.1                               0.723±0.024  0.672±0.034   
          0.3                               0.699±0.016  0.708±0.023   
          0.5                               0.862±0.003  0.867±0.004   

                                                   VREx          RVP  \
es_method corr_label_train_corrupt_mean                                
train     0.1                               0.343±0.041  0.458±0.014   
          0.3                               0.703±0.023  0.705±0.017   
          0.5                               0.861±0.005  0.856±0.009   
val       0.1                               0.707±0.043  0.653±0.070   
          0.3                               0.695±0.018  0.708±0.025   
          0.5                               0.861±0.005  0.854±0.007   

                                                    IGA        CORAL  \
es_method corr_label_train_corrupt_mean                                
train     0.1                               0.488±0.157  0.331±0.032   
          0.3                               0.707±0.012  0.688±0.035   
          0.5                               0.794±0.072  0.863±0.011   
val       0.1                               0.714±0.010  0.707±0.030   
          0.3                               0.740±0.026  0.723±0.017   
          0.5                               0.826±0.054  0.860±0.009   

                                                   MLDG  
es_method corr_label_train_corrupt_mean                  
train     0.1                               0.317±0.043  
          0.3                               0.669±0.032  
          0.5                               0.867±0.007  
val       0.1                               0.716±0.017  
          0.3                               0.733±0.012  
          0.5                               0.864±0.008

## eICU CorrNoise

In [53]:
path = Path('/scratch/ssd001/home/haoran/domainbed/eICUCorrNoise')
lst = load_saved_data(path) 

In [54]:
keep = ['es_roc', 'test_results/South_roc',  
        'test_results/South_tpr_gap_thres_wb',
        'model_hparams/eicu_architecture', 'args/algorithm', 'args/es_method', 'args/hparams_seed',
       'args/trial_seed', 'model_hparams/corr_noise_train_corrupt_mean', 'model_hparams/corr_noise_train_corrupt_dist']

df = extract_stats(lst, keep)

In [55]:
distinct_hparams = ['algorithm', 'es_method', 'corr_noise_train_corrupt_mean', 'corr_noise_train_corrupt_dist']
eval_metric = 'es_roc'
other_metrics = ['South_roc', 'South_tpr_gap_thres_wb']

In [56]:
df = agg_and_group(df, distinct_hparams, eval_metric, other_metrics)

In [57]:
df = df[df.corr_noise_train_corrupt_mean.isin([1.0, 2.0])]

In [58]:
print('South ROC')
pivot(df, ['South_roc'], ['es_method', 'corr_noise_train_corrupt_mean', 'corr_noise_train_corrupt_dist'])

South ROC


Oracle  \
es_method corr_noise_train_corrupt_mean corr_noise_train_corrupt_dist                
train     1.0                           0.5                            0.956±0.004   
                                        1.0                            0.956±0.004   
          2.0                           0.5                            0.956±0.004   
                                        1.0                            0.956±0.004   
val       1.0                           0.5                            0.956±0.004   
                                        1.0                            0.956±0.004   
          2.0                           0.5                            0.956±0.004   
                                        1.0                            0.956±0.004   

                                                                               ERM  \
es_method corr_noise_train_corrupt_mean corr_noise_train_corrupt_dist                
train     1.0                           0.5                            0.371±0.036   
                                        1.0                            0.523±0.047   
          2.0                           0.5                            0.208±0.030   
                                        1.0                            0.248±0.024   
val       1.0                           0.5                            0.396±0.055   
                                        1.0                            0.574±0.058   
          2.0                           0.5                            0.439±0.115   
                                        1.0                            0.262±0.020   

                                                                          GroupDRO  \
es_method corr_noise_train_corrupt_mean corr_noise_train_corrupt_dist                
train     1.0                           0.5                            0.389±0.040   
                                        1.0                            0.562±0.020   
          2.0                           0.5                            0.205±0.017   
                                        1.0                            0.247±0.029   
val       1.0                           0.5                            0.438±0.042   
                                        1.0                            0.625±0.055   
          2.0                           0.5                            0.390±0.109   
                                        1.0                            0.263±0.019   

                                                                               IRM  \
es_method corr_noise_train_corrupt_mean corr_noise_train_corrupt_dist                
train     1.0                           0.5                            0.373±0.038   
                                        1.0                            0.525±0.033   
          2.0                           0.5                            0.195±0.021   
                                        1.0                            0.242±0.017   
val       1.0                           0.5                            0.400±0.046   
                                        1.0                            0.547±0.056   
          2.0                           0.5                            0.292±0.049   
                                        1.0                            0.305±0.041   

                                                                              VREx  \
es_method corr_noise_train_corrupt_mean corr_noise_train_corrupt_dist                
train     1.0                           0.5                            0.402±0.028   
                                        1.0                            0.593±0.023   
          2.0                           0.5                            0.197±0.022   
                                        1.0                            0.238±0.041   
val       1.0                           0.5                            0.416±0.022   
                              